In [24]:
pip install xmltodict

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import os
import xmltodict
import glob

# Define paths to your dataset
images_path = 'Dataset/cardboard_images'
annotations_path = 'Dataset/cardboard_annotation'

# Class name mapping (YOLO format uses numeric IDs, so map 'cardboard' to 0)
class_name_to_id = {'cardboard': 0}

def convert_voc_to_yolo(xml_file, output_txt_path, image_size):
    # Parse XML annotation
    with open(xml_file) as f:
        xml_content = xmltodict.parse(f.read())

    image_width = image_size[0]
    image_height = image_size[1]

    # Check if the XML file contains any object
    if 'object' not in xml_content['annotation']:
        return

    # Open the output text file for YOLO labels
    with open(output_txt_path, 'w') as txt_file:
        # Handle single or multiple objects
        objects = xml_content['annotation']['object']
        if isinstance(objects, dict):
            objects = [objects]  # Convert single object to list for consistency

        for obj in objects:
            class_name = obj['name']
            class_id = class_name_to_id.get(class_name)

            if class_id is None:
                print(f"Warning: Class '{class_name}' not found in class map.")
                continue

            # Extract the bounding box and normalize coordinates
            bbox = obj['bndbox']
            xmin = int(bbox['xmin'])
            ymin = int(bbox['ymin'])
            xmax = int(bbox['xmax'])
            ymax = int(bbox['ymax'])

            # YOLO format requires normalized center coordinates and width/height
            x_center = ((xmin + xmax) / 2) / image_width
            y_center = ((ymin + ymax) / 2) / image_height
            width = (xmax - xmin) / image_width
            height = (ymax - ymin) / image_height

            # Write to the txt file in YOLO format
            txt_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Process all XML annotations and convert them to YOLO format
def convert_dataset(annotations_folder, images_folder):
    for split in ['train', 'val']:
        xml_files = glob.glob(f'{annotations_folder}/{split}/*.xml')
        os.makedirs(f'{images_folder}/labels/{split}', exist_ok=True)

        for xml_file in xml_files:
            # Extract image size from the XML annotation
            with open(xml_file) as f:
                xml_content = xmltodict.parse(f.read())
                image_width = int(xml_content['annotation']['size']['width'])
                image_height = int(xml_content['annotation']['size']['height'])
            # Create the output txt path
            image_filename = os.path.basename(xml_file).replace('.xml', '.jpg')
            txt_output_path = f'{images_folder}/labels/{split}/{image_filename.replace(".jpg", ".txt")}'

            # Convert and save the YOLO label
            convert_voc_to_yolo(xml_file, txt_output_path, (image_width, image_height))

# Run the conversion for the train and val sets
convert_dataset(annotations_path, images_path)


In [39]:
import sys
print(sys.executable)
!{sys.executable} -m pip install albumentations

c:\Program Files\Python38\python.exe


'c:\Program' is not recognized as an internal or external command,
operable program or batch file.


In [37]:
import albumentations as A
import cv2
import os
import shutil
import numpy as np

# Define augmentation pipeline
augmentor = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=15, p=0.5),  # Rotate by ±15 degrees
    A.Resize(640, 640)  # Resize all images to 640x640
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Folder paths
input_images_folder = 'Dataset/cardboard_images/train'  # Path to your training images
input_labels_folder = 'Dataset/cardboard_images/labels/train'  # Path to your training labels
output_images_folder = 'Augmented_Data/images/train'  # Augmented images folder
output_labels_folder = 'Augmented_Data/labels/train'  # Augmented labels folder

# Create output directories if they don't exist
os.makedirs(output_images_folder, exist_ok=True)
os.makedirs(output_labels_folder, exist_ok=True)

# Function to read YOLO labels
def read_yolo_label(label_path):
    with open(label_path, 'r') as f:
        bboxes = []
        class_labels = []
        for line in f.readlines():
            line = line.strip().split()
            class_id = int(line[0])
            bbox = [float(x) for x in line[1:]]
            bboxes.append(bbox)
            class_labels.append(class_id)
    return bboxes, class_labels

# Function to save augmented YOLO labels
def save_yolo_label(label_path, bboxes, class_labels):
    with open(label_path, 'w') as f:
        for bbox, class_id in zip(bboxes, class_labels):
            bbox_str = ' '.join([str(x) for x in bbox])
            f.write(f'{class_id} {bbox_str}\n')

# Loop through all images and labels in the dataset and apply augmentations
for image_file in os.listdir(input_images_folder):
    if image_file.endswith('.jpg'):
        # Read image
        img_path = os.path.join(input_images_folder, image_file)
        image = cv2.imread(img_path)

        # Read corresponding label
        label_file = image_file.replace('.jpg', '.txt')
        label_path = os.path.join(input_labels_folder, label_file)
        bboxes, class_labels = read_yolo_label(label_path)

        # Apply augmentation
        augmented = augmentor(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']
        augmented_class_labels = augmented['class_labels']

        # Save augmented image and label
        output_image_path = os.path.join(output_images_folder, f'aug_{image_file}')
        output_label_path = os.path.join(output_labels_folder, f'aug_{label_file}')
        cv2.imwrite(output_image_path, augmented_image)
        save_yolo_label(output_label_path, augmented_bboxes, augmented_class_labels)

print("Data augmentation completed.")


ModuleNotFoundError: No module named 'albumentations'